In [1]:
import pandas as pd
population=pd.read_csv('county_population.csv',index_col='US County')
candidate=pd.read_csv('president_county_candidate.csv')


In [2]:
population

,Population
US County,
".Autauga County, Alabama",55869
".Baldwin County, Alabama",223234
".Barbour County, Alabama",24686
".Bibb County, Alabama",22394
".Blount County, Alabama",57826
...,...
".Sweetwater County, Wyoming",42343
".Teton County, Wyoming",23464
".Uinta County, Wyoming",20226


In [3]:
candidate

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True
...,...,...,...,...,...,...
32172,Arizona,Maricopa County,Write-ins,WRI,1331,False
32173,Arizona,Mohave County,Donald Trump,REP,78535,True
32174,Arizona,Mohave County,Joe Biden,DEM,24831,False
32175,Arizona,Mohave County,Jo Jorgensen,LIB,1302,False


任务一大致思路：将投票表索引的索引转化为人口表的形式，使用分组计算每个县的总投票人数，合并到人口表，再使用逻辑判断计算目标数

In [4]:
count=candidate.set_index(['state','county'])
count.index=count.index.map(lambda x:f'.{x[1]}, {x[0]}')
count=count.groupby(count.index)['total_votes'].sum().to_frame()

In [5]:
pop_count= population.merge(count,left_on='US County',right_on=count.index, how='left')
sum(pop_count['Population']>pop_count['total_votes']*2)

1609

大致思路：先通过分组，计算每个州的票数，然后再变形

In [7]:
count_state=candidate.groupby(['state','candidate'])['total_votes'].sum().to_frame()
count_state=count_state.reset_index()
count_state.pivot(index='state', columns='candidate', values='total_votes')

candidate,None of these candidates,Write-ins,Alyson Kennedy,Bill Hammons,Blake Huber,Brian Carroll,Brock Pierce,Brooke Paige,Christopher LaFontaine,Connie Gammon,...,Mark Charles,Phil Collins,President Boddie,Princess Jacob-Fambro,Richard Duncan,Ricki Sue King,Rocky De La Fuente,Sheila Samm Tittle,Tom Hoefling,Zachary Scalf
state,,,,,,,,,,,,,,,,,,,,,
Alabama,NaN,7312.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alaska,NaN,34210.0,NaN,NaN,NaN,NaN,825.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,318.0,NaN,NaN,NaN
Arizona,NaN,2032.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arkansas,NaN,NaN,NaN,NaN,NaN,1713.0,2141.0,NaN,NaN,1475.0,...,NaN,2812.0,NaN,NaN,NaN,NaN,1321.0,NaN,NaN,NaN
California,NaN,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,60155.0,NaN,NaN,NaN
Colorado,NaN,0.0,354.0,2730.0,355.0,2516.0,572.0,NaN,NaN,NaN,...,2011.0,568.0,NaN,495.0,NaN,NaN,636.0,NaN,NaN,NaN
Connecticut,NaN,544.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Delaware,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
District of Columbia,NaN,3137.0,NaN,NaN,NaN,NaN,693.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


自定义描述函数，分组过滤

In [9]:
def select(x):
    def inner_select(inner_x):
        Total = inner_x.total_votes.sum()
        Biden = inner_x.query('candidate=="Joe Biden"').total_votes.sum()
        Trump = inner_x.query('candidate=="Donald Trump"').total_votes.sum()
        return (Biden-Trump)/Total
    res = x.groupby('county')[['candidate','total_votes']].apply(inner_select)
candidate.groupby('state').filter(select).state.unique()

array(['Delaware', 'District of Columbia', 'Hawaii', 'Massachusetts',
       'New Jersey', 'Rhode Island', 'Vermont', 'California',
       'Connecticut'], dtype=object)